# ACL Anthology

In [1]:
# def buscar_artigo(query, base_dados):
#     # TODO: integração com a API da base de dados
#     # Por exemplo, usando requests.get() se a API for RESTful
#     resposta = requests.get(f"API_URL?query={query}&database={base_dados}")
    
#     if resposta.status_code == 200:
#         dados_artigo = resposta.json()  # Exemplo, depende do formato retornado pela API
#         bib_entry = formatar_citacao_bib(dados_artigo)
#         return bib_entry
#     else:
#         return "Não foi possível encontrar o artigo."

# def formatar_citacao_bib(dados_artigo):
#     # Formatar os dados do artigo em uma entrada .bib
#     # Isso é um exemplo, a estrutura real depende dos dados retornados pela API
#     return f"@InProceedings{{ {dados_artigo['id']},\n" \
#            f"  title={{ {dados_artigo['title']} }},\n" \
#            f"  author={{ {', '.join(dados_artigo['authors'])} }},\n" \
#            f"  year={{ {dados_artigo['year']} }},\n" \
#            f"  booktitle={{ {dados_artigo['booktitle']} }},\n" \
#            f"  url_papwc={{ {dados_artigo.get('url_papwc', '')} }},\n" \
#            f"  url_arxiv={{ {dados_artigo.get('url_arxiv', '')} }},\n" \
#            f"}}\n\n" \
#            f"Abstract: \n{dados_artigo['abstract']}\n"


# # Exemplo de uso
# # bib_entry = buscar_artigo("Exemplo de Query", "base_dados_exemplo")
# # print(bib_entry)

In [2]:
import requests
import gzip
import re

def contar_entradas_bib_no_arquivo_compactado(caminho_arquivo_bib_gz):
    padrao_entrada_bib = re.compile(r'@[\w]+{')
    contador = 0

    with gzip.open(caminho_arquivo_bib_gz, 'rt', encoding='utf-8') as arquivo:
        for linha in arquivo:
            if padrao_entrada_bib.match(linha):
                contador += 1

    return contador

def extrair_e_buscar_no_bib(query, caminho_arquivo_bib):
    with gzip.open(caminho_arquivo_bib, 'rt', encoding='utf-8') as arquivo:
        conteudo = arquivo.read()

        # Busca simplificada; pode ser necessário ajustar para ser mais específica
        padrao = re.compile(r'@inproceedings{.*?'+ re.escape(query) + r'.*?}', re.DOTALL)
        resultado = padrao.findall(conteudo)

        return resultado

def busca_simples(query, caminho_arquivo_bib_gz, ano_inicio=None, ano_fim=None):
    padrao = re.compile(r'@[\w]+\{.*?\}', re.DOTALL)
    padrao_query = re.compile(re.escape(query), re.IGNORECASE)
    padrao_ano = re.compile(r'year = "(\d{4})"')

    resultados = []

    with gzip.open(caminho_arquivo_bib_gz, 'rt', encoding='utf-8') as arquivo:
        conteudo = arquivo.read()
        entradas = padrao.findall(conteudo)

        for entrada in entradas:
            if padrao_query.search(entrada):
                ano_match = padrao_ano.search(entrada)
                if ano_match:
                    ano = int(ano_match.group(1))
                    if (ano_inicio is None or ano >= ano_inicio) and (ano_fim is None or ano <= ano_fim):
                        resultados.append(entrada)

    return resultados

def busca_avancada(query, caminho_arquivo_bib_gz, ano_inicio=None, ano_fim=None):
    padrao = re.compile(r'@[\w]+\{.*?\}', re.DOTALL)
    padrao_ano = re.compile(r'year = "(\d{4})"')

    # Divide a query em termos e operadores
    termos_operadores = re.split(r'\s+(AND|OR|NOT)\s+', query)
    termos = [termos_operadores[i] for i in range(len(termos_operadores)) if i % 2 == 0]
    operadores = [termos_operadores[i] for i in range(len(termos_operadores)) if i % 2 != 0]
    
    # Compila regex para cada termo
    padroes_termos = [re.compile(re.escape(termo), re.IGNORECASE) for termo in termos]

    resultados = []

    with gzip.open(caminho_arquivo_bib_gz, 'rt', encoding='utf-8') as arquivo:
        conteudo = arquivo.read()
        entradas = padrao.findall(conteudo)

        for entrada in entradas:
            ano_match = padrao_ano.search(entrada)
            ano = int(ano_match.group(1)) if ano_match else None
            if ano_inicio and ano and ano < ano_inicio:
                continue
            if ano_fim and ano and ano > ano_fim:
                continue

            # Avalia se a entrada corresponde aos termos e operadores
            match = True
            for i, padrao_termo in enumerate(padroes_termos):
                termo_presente = padrao_termo.search(entrada) is not None
                if operadores and i < len(operadores):
                    operador = operadores[i]
                    if operador == "AND":
                        match = match and termo_presente
                    elif operador == "OR":
                        match = match or termo_presente
                    elif operador == "NOT":
                        match = match and not termo_presente
                else:
                    match = match and termo_presente

            if match:
                resultados.append(entrada)

    return resultados

def listar_titulos(entradas_bib):
    padrao_titulo = re.compile(r'title = {(.+?)}')
    titulos = []

    for entrada in entradas_bib:
        titulo_match = padrao_titulo.search(entrada)
        if titulo_match:
            titulos.append(titulo_match.group(1))

    return titulos

In [3]:
folder  = '../data/'
acl_bib = 'anthology+abstracts.bib'
acl_bib_gz = 'anthology+abstracts.bib.gz'

total_entradas = contar_entradas_bib_no_arquivo_compactado(folder+acl_bib_gz)
print(f"{total_entradas} publicações no acervo da ACL Anthology")

89047 publicações no acervo da ACL Anthology


In [6]:
termo_simples = "Innovation"
ano_inicio = 1990
ano_fim = 2024
resultados_simples = busca_simples(termo_simples, folder+acl_bib_gz, ano_inicio, ano_fim)
print(f"{len(resultados_simples)} publicações no acervo da ACL Anthology com termo '{termo_simples}' no período {ano_inicio}-{ano_fim}")


28 publicações no acervo da ACL Anthology com termo 'Innovation' no período 1990-2024


In [ ]:
# from pprint import pprint
# for resultado in resultados_simples:
#     pprint(resultado, width=125)

In [10]:
query_booleana = "innovation AND model"
resultados = busca_simples(query_booleana, folder+acl_bib_gz, ano_inicio=1990, ano_fim=2024)
print(f"{len(resultados)} publicações no acervo da ACL Anthology com termo '{query_booleana}'")

0 publicações no acervo da ACL Anthology com termo 'innovation AND model'


## Pré-print Repositories

In [ ]:
import requests
import numpy as np
from pprint import pprint
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
def scrape_subject_areas(url):
    # Enviar uma solicitação GET para a página
    response = requests.get(url)

    # Dicionário para armazenar as áreas de assunto
    subject_areas_dict = {}

    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Analise o conteúdo da página com BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontre o elemento div que contém as áreas de assunto
        subject_areas_div = soup.find('div', class_='item-list highwire-views-col-list highwire-views-cols-3 clearfix')

        # Verifique se o elemento foi encontrado
        if subject_areas_div:
            # Encontre todas as tags 'a' dentro do div
            subject_links = subject_areas_div.find_all('a')
            
            # Itere sobre as tags 'a' para extrair as áreas de assunto e seus URLs
            for link in subject_links:
                area_name = link.text.strip()
                area_url = link['href']
                subject_areas_dict[area_name] = area_url

        # Exiba o dicionário resultante
        # print(subject_areas_dict)

    return subject_areas_dict

In [ ]:
url_bioRxiv = 'https://www.biorxiv.org/'
areas_dict = scrape_subject_areas(url_bioRxiv)
print(f'{len(areas_dict)} coleções de áreas')

if areas_dict:
    for area, area_url in areas_dict.items():
        print(f'   Área: {area:40} URL: {area_url}')
else:
    print('Seção de áreas de assunto não encontrada.')

In [ ]:
def fetch_content_summary_statistics(interval='m', format_='json'):
    # Construa a URL do endpoint
    url = f'https://api.biorxiv.org/sum/{interval}/{format_}'
    
    # Faça a solicitação GET
    response = requests.get(url)
    
    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Erro ao buscar estatísticas de resumo de conteúdo. Código de status: {response.status_code}")
        return None

def fetch_usage_summary_statistics(interval='m', format_='json'):
    # Construa a URL do endpoint
    url = f'https://api.biorxiv.org/usage/{interval}/{format_}'
    
    # Faça a solicitação GET
    response = requests.get(url)
    
    # Verifique se a solicitação foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Erro ao buscar estatísticas de uso. Código de status: {response.status_code}")
        return None

In [ ]:
# Exemplo de uso das funções
content_summary_data = fetch_content_summary_statistics()
usage_summary_data   = fetch_usage_summary_statistics()

In [ ]:
## Exibir os resultados
# print("Estatísticas de Resumo de Conteúdo:")
# pprint(content_summary_data, width=125)

In [ ]:
# print("Estatísticas de Uso:")
# pprint(usage_summary_data, width=125)

In [ ]:
def plot_usage_statistics(data, width=1800, height=800):
    # Extrair os valores relevantes dos dados
    months = [entry['month'] for entry in data['bioRxiv content statistics']]
    abstract_cumulative = [np.round(int(entry['abstract_cumulative'])/1000000, 2) for entry in data['bioRxiv content statistics']]
    abstract_views = [np.round(int(entry['abstract_views'])/1000000, 2) for entry in data['bioRxiv content statistics']]

    # Criar um DataFrame com os dados
    import pandas as pd
    df = pd.DataFrame({'Mês': months, 'Resumo Cumulativo': abstract_cumulative, 'Visualizações de Resumo': abstract_views})

    # Criar um gráfico com dois eixos y
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Adicionar barras para 'Resumo Cumulativo'
    fig.add_trace(go.Bar(x=df['Mês'], y=df['Resumo Cumulativo'], name='Resumo Cumulativo', text=df['Resumo Cumulativo'], textposition='outside'))

    # Adicionar linhas para 'Visualizações de Resumo'
    fig.add_trace(go.Scatter(x=df['Mês'], y=df['Visualizações de Resumo'], mode='lines+markers+text', name='Visualizações de Resumo', textposition='top center', yaxis='y2'))

    # Configurar eixos
    max_value_primary = max(df['Resumo Cumulativo'].max()+30, df['Visualizações de Resumo'].max())
    ratio = 50  # Proporção desejada
    max_value_secondary = max_value_primary / ratio

    fig.update_layout(
        title='Estatísticas de uso do bioRxiv',
        xaxis_title='Mês',
        yaxis_title='Milhões de resumos acumulados',
        yaxis2_title='Milhões de visualizações por mês',
        legend=dict(x=0.02, y=0.96),
        width=width,
        height=height,
        yaxis=dict(range=[0, max_value_primary]),
        yaxis2=dict(range=[0, max_value_secondary])
    )

    # Configurar o tamanho da fonte para os rótulos de dados
    fig.update_traces(
        textfont=dict(size=24),  # Tamanho da fonte para os rótulos de dados
        selector=dict(type='bar', name='Resumo Cumulativo')  # Seletor para as barras
    )

    fig.update_traces(
        textfont=dict(size=40),  # Tamanho da fonte para os rótulos de dados
        selector=dict(type='scatter', name='Visualizações de Resumo')  # Seletor para as linhas
    )

    # Exibir o gráfico
    fig.show()

def plot_content_statistics(content_data, width=1800, height=800):
    # Extrair os valores relevantes dos dados
    months = [entry['month'] for entry in content_data['bioRxiv content statistics']]
    new_papers = [entry['new_papers'] for entry in content_data['bioRxiv content statistics']]
    revised_papers = [entry['revised_papers'] for entry in content_data['bioRxiv content statistics']]
    new_papers_cumulative = [entry['new_papers_cumulative'] for entry in content_data['bioRxiv content statistics']]
    revised_papers_cumulative = [entry['revised_papers_cumulative'] for entry in content_data['bioRxiv content statistics']]

    # Criar um DataFrame com os dados
    import pandas as pd
    df = pd.DataFrame({'Mês': months,
                        'Artigos novos no mês': new_papers, 
                        'Artigos revisados no mês':revised_papers, 
                        'Total de Artigos acumulado':new_papers_cumulative , 
                        'Total de Artigos revisados':revised_papers_cumulative})

    # Criar um gráfico com dois eixos y
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Adicionar barras para total acumulado de artigos
    fig.add_trace(go.Bar(x=df['Mês'], y=df['Total de Artigos acumulado'], name='Total de Artigos acumulado', text=df['Total de Artigos acumulado'], textposition='outside'))
    fig.add_trace(go.Bar(x=df['Mês'], y=df['Total de Artigos revisados'], name='Total de Artigos revisados', text=df['Total de Artigos revisados'], textposition='outside'))

    # Configurar eixo principal
    fig.update_yaxes(title_text='Quantidade de Artigos', secondary_y=False)

    # Adicionar linhas para versões acumuladas no eixo secundário
    fig.add_trace(go.Scatter(x=df['Mês'], y=df['Artigos novos no mês'], mode='lines+markers+text', name='Artigos novos no mês', textposition='top center', yaxis='y2'))
    fig.add_trace(go.Scatter(x=df['Mês'], y=df['Artigos revisados no mês'], mode='lines+markers+text', name='Artigos revisados no mês', textposition='top center', yaxis='y2'))

    # Configurar eixos
    max_value_primary = max(df['Total de Artigos acumulado'].max()+30, df['Total de Artigos revisados'].max())
    ratio = 50  # Proporção desejada
    max_value_secondary = max_value_primary / ratio

    fig.update_layout(
        title='Estatísticas de conteúdo no bioRxiv',
        xaxis_title='Mês',
        yaxis_title='Quantidade acumulada de artigos',
        yaxis2_title='Quantidades de artigos em cada mês',
        legend=dict(x=0.02, y=0.96),
        width=width,
        height=height,
        yaxis=dict(range=[0, max_value_primary]),
        yaxis2=dict(range=[0, max_value_secondary])
    )

    # Configurar o tamanho da fonte para os rótulos de dados
    fig.update_traces(
        textfont=dict(size=24),
        selector=dict(type='bar', name='Quantidade de novos papers')  # Seletor para as barras
    )

    fig.update_traces(
        textfont=dict(size=40),
        selector=dict(type='scatter', name='Quantidade de papers revisados')  # Seletor para as linhas
    )
    
    # fig.update_yaxes(overlaying='y2', side='right')

    # Exibir o gráfico
    fig.show()

In [ ]:
plot_content_statistics(content_summary_data)
plot_usage_statistics(usage_summary_data)

In [ ]:
## Matplotlib
# def plot_usage_statistics_simple(data):
#     # Extrair os valores relevantes dos dados
#     months = [entry['month'] for entry in data['bioRxiv content statistics']]
#     abstract_views = [int(entry['abstract_views']) for entry in data['bioRxiv content statistics']]
#     pdf_downloads = [int(entry['pdf_downloads']) for entry in data['bioRxiv content statistics']]

#     # Criar um gráfico de linha
#     plt.figure(figsize=(18, 9))
#     plt.plot(months, abstract_views, label='Visualizações de Resumo de Conteúdo', marker='o', linestyle='-', color='b')
#     plt.plot(months, pdf_downloads, label='Downloads de PDF', marker='s', linestyle='--', color='r')

#     # Configurar rótulos e título
#     plt.xlabel('Mês')
#     plt.ylabel('Contagem')
#     plt.title('Estatísticas de Uso do bioRxiv')
#     plt.xticks(rotation=45, fontsize=8)  # Rotação dos rótulos do eixo x para melhor legibilidade
#     plt.grid(True)

#     # Adicionar legenda
#     plt.legend()

#     # Exibir o gráfico
#     plt.tight_layout()
#     plt.show()

# # Exemplo de uso com os dados fornecidos
# usage_data = fetch_usage_summary_statistics()

# plot_usage_statistics(usage_data)